# EE 451: Communications Systems
## Lecture 29 - Modern Wireless Systems: Link Budgets, WiFi, Cellular & SDR

### Learning Objectives

By the end of this notebook, you will be able to:

1. Analyze WiFi standards evolution (802.11a/g/n/ac/ax/be)
2. Compare LTE and 5G NR architecture and capabilities
3. Explain MIMO and beamforming in modern wireless
4. Calculate link budgets for WiFi and cellular systems
5. Understand mmWave challenges in 5G
6. Apply Friis equation for path loss calculations

---

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import constants

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Physical constants
c = constants.c  # Speed of light (m/s)

print("Libraries imported successfully!")
print(f"Speed of light: c = {c:.3e} m/s")

## Part 1: Link Budget Fundamentals

A **link budget** calculates the received signal power in a wireless system.

### Friis Transmission Equation
$$P_r = P_t + G_t + G_r - L_p$$

(in dB scale)

where:
- $P_r$ = Received power (dBm)
- $P_t$ = Transmitted power (dBm)
- $G_t$ = Transmit antenna gain (dBi)
- $G_r$ = Receive antenna gain (dBi)
- $L_p$ = Path loss (dB)

### Free Space Path Loss (FSPL)
$$L_p = 20\log_{10}(d) + 20\log_{10}(f) + 32.45$$

where:
- $d$ = Distance (km)
- $f$ = Frequency (MHz)
- Result in dB

In [ ]:
# Define link budget functions
def friis_path_loss(distance_km, frequency_MHz):
    """
    Calculate free space path loss using Friis equation
    
    Parameters:
    - distance_km: Distance in kilometers
    - frequency_MHz: Frequency in MHz
    
    Returns:
    - Path loss in dB
    """
    L_p = 20 * np.log10(distance_km) + 20 * np.log10(frequency_MHz) + 32.45
    return L_p

def link_budget(Pt_dBm, Gt_dBi, Gr_dBi, distance_km, frequency_MHz, margin_dB=0):
    """
    Calculate received power using link budget equation
    
    Parameters:
    - Pt_dBm: Transmit power (dBm)
    - Gt_dBi: Transmit antenna gain (dBi)
    - Gr_dBi: Receive antenna gain (dBi)
    - distance_km: Distance (km)
    - frequency_MHz: Frequency (MHz)
    - margin_dB: Additional losses/margin (dB)
    
    Returns:
    - Pr_dBm: Received power (dBm)
    - L_p: Path loss (dB)
    """
    L_p = friis_path_loss(distance_km, frequency_MHz)
    Pr_dBm = Pt_dBm + Gt_dBi + Gr_dBi - L_p - margin_dB
    return Pr_dBm, L_p

# Example: WiFi link budget
print("Example 1: WiFi Link Budget (2.4 GHz)")
print("═" * 70)

# WiFi parameters
Pt_wifi = 20  # dBm (100 mW)
Gt_wifi = 2   # dBi (omnidirectional antenna)
Gr_wifi = 0   # dBi (laptop antenna)
freq_wifi = 2400  # MHz (2.4 GHz)
distance_wifi = 0.05  # km (50 meters)

Pr_wifi, Lp_wifi = link_budget(Pt_wifi, Gt_wifi, Gr_wifi, distance_wifi, freq_wifi)

print(f"Transmit power: {Pt_wifi} dBm")
print(f"TX antenna gain: {Gt_wifi} dBi")
print(f"RX antenna gain: {Gr_wifi} dBi")
print(f"Frequency: {freq_wifi} MHz")
print(f"Distance: {distance_wifi*1000} m")
print(f"\nPath loss: {Lp_wifi:.1f} dB")
print(f"Received power: {Pr_wifi:.1f} dBm")
print("═" * 70)

# Example: Cellular link budget
print("\nExample 2: Cellular Link Budget (800 MHz)")
print("═" * 70)

Pt_cell = 43  # dBm (20 W base station)
Gt_cell = 15  # dBi (base station directional antenna)
Gr_cell = 0   # dBi (phone antenna)
freq_cell = 800  # MHz
distance_cell = 2  # km

Pr_cell, Lp_cell = link_budget(Pt_cell, Gt_cell, Gr_cell, distance_cell, freq_cell)

print(f"Transmit power: {Pt_cell} dBm")
print(f"TX antenna gain: {Gt_cell} dBi")
print(f"RX antenna gain: {Gr_cell} dBi")
print(f"Frequency: {freq_cell} MHz")
print(f"Distance: {distance_cell} km")
print(f"\nPath loss: {Lp_cell:.1f} dB")
print(f"Received power: {Pr_cell:.1f} dBm")
print("═" * 70)

## Part 2: Path Loss vs Frequency and Distance

Path loss increases with:
1. **Distance**: 20 dB/decade (doubles every 10×)
2. **Frequency**: 20 dB/decade

Let's visualize these relationships.

In [ ]:
# Path loss vs distance for different frequencies
distances = np.logspace(-2, 1, 100)  # 0.01 to 10 km
frequencies = [900, 2400, 5000, 28000, 60000]  # MHz (various bands)
freq_labels = ['900 MHz (Cellular)', '2.4 GHz (WiFi)', '5 GHz (WiFi)', 
               '28 GHz (5G mmWave)', '60 GHz (WiGig)']

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Path loss vs distance
colors = plt.cm.viridis(np.linspace(0.2, 0.9, len(frequencies)))
for freq, label, color in zip(frequencies, freq_labels, colors):
    path_loss = friis_path_loss(distances, freq)
    axes[0].semilogx(distances, path_loss, linewidth=2, label=label, color=color)

axes[0].set_xlabel('Distance (km)', fontsize=12)
axes[0].set_ylabel('Path Loss (dB)', fontsize=12)
axes[0].set_title('Free Space Path Loss vs Distance', fontsize=14)
axes[0].grid(True, which='both', alpha=0.3)
axes[0].legend(fontsize=9)
axes[0].set_xlim(0.01, 10)

# Path loss vs frequency (at fixed distance)
freq_range = np.logspace(2, 5, 100)  # 100 MHz to 100 GHz
distances_fixed = [0.01, 0.1, 1, 10]  # km
distance_labels = ['10 m', '100 m', '1 km', '10 km']

colors2 = plt.cm.plasma(np.linspace(0.2, 0.9, len(distances_fixed)))
for dist, label, color in zip(distances_fixed, distance_labels, colors2):
    path_loss = friis_path_loss(dist, freq_range)
    axes[1].semilogx(freq_range, path_loss, linewidth=2, label=label, color=color)

# Mark common frequency bands
bands = [(900, 'Cellular'), (2400, 'WiFi 2.4G'), (5000, 'WiFi 5G'), (28000, '5G mmWave')]
for freq, name in bands:
    axes[1].axvline(x=freq, color='gray', linestyle='--', linewidth=1, alpha=0.5)
    axes[1].text(freq, 150, name, rotation=90, fontsize=8, alpha=0.7)

axes[1].set_xlabel('Frequency (MHz)', fontsize=12)
axes[1].set_ylabel('Path Loss (dB)', fontsize=12)
axes[1].set_title('Free Space Path Loss vs Frequency', fontsize=14)
axes[1].grid(True, which='both', alpha=0.3)
axes[1].legend(fontsize=9)
axes[1].set_xlim(100, 1e5)

plt.tight_layout()
plt.show()

# Calculate path loss difference
print("\nPath Loss Comparison:")
print("═" * 70)
print("At 100 m distance:")
for freq, label in zip(frequencies[:4], freq_labels[:4]):
    pl = friis_path_loss(0.1, freq)
    print(f"  {label:<25}: {pl:.1f} dB")

pl_900 = friis_path_loss(0.1, 900)
pl_2400 = friis_path_loss(0.1, 2400)
pl_28000 = friis_path_loss(0.1, 28000)

print(f"\n2.4 GHz vs 900 MHz: +{pl_2400 - pl_900:.1f} dB more loss")
print(f"28 GHz vs 2.4 GHz: +{pl_28000 - pl_2400:.1f} dB more loss")
print(f"28 GHz vs 900 MHz: +{pl_28000 - pl_900:.1f} dB more loss")
print("\nConclusion: Higher frequency = Higher path loss = Shorter range")
print("═" * 70)

## Part 3: WiFi Standards Evolution

WiFi has evolved significantly over the years, with each generation improving:
- **Data rates** (higher modulation orders)
- **Spectral efficiency** (MIMO, wider channels)
- **Capacity** (MU-MIMO, OFDMA)

Let's analyze the progression.

In [ ]:
# WiFi standards comparison
wifi_standards = {
    '802.11a': {'year': 1999, 'freq': '5 GHz', 'max_rate': 54, 'mod': '64-QAM', 'mimo': '1x1'},
    '802.11g': {'year': 2003, 'freq': '2.4 GHz', 'max_rate': 54, 'mod': '64-QAM', 'mimo': '1x1'},
    '802.11n (WiFi 4)': {'year': 2009, 'freq': '2.4/5 GHz', 'max_rate': 600, 'mod': '64-QAM', 'mimo': '4x4'},
    '802.11ac (WiFi 5)': {'year': 2014, 'freq': '5 GHz', 'max_rate': 6933, 'mod': '256-QAM', 'mimo': '8x8'},
    '802.11ax (WiFi 6)': {'year': 2019, 'freq': '2.4/5/6 GHz', 'max_rate': 9608, 'mod': '1024-QAM', 'mimo': '8x8'},
    '802.11be (WiFi 7)': {'year': 2024, 'freq': '2.4/5/6 GHz', 'max_rate': 46000, 'mod': '4096-QAM', 'mimo': '16x16'},
}

# Extract data for plotting
years = [s['year'] for s in wifi_standards.values()]
rates = [s['max_rate'] for s in wifi_standards.values()]
names = list(wifi_standards.keys())

# Plot evolution
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Data rate evolution
colors = plt.cm.viridis(np.linspace(0.2, 0.9, len(years)))
bars = axes[0].bar(range(len(names)), rates, color=colors)
axes[0].set_xticks(range(len(names)))
axes[0].set_xticklabels(names, rotation=45, ha='right')
axes[0].set_ylabel('Max Data Rate (Mbps)', fontsize=12)
axes[0].set_title('WiFi Standards Evolution: Data Rates', fontsize=14)
axes[0].set_yscale('log')
axes[0].grid(True, alpha=0.3, axis='y')

# Add values on bars
for i, (bar, rate) in enumerate(zip(bars, rates)):
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height*1.1,
                f'{rate} Mbps', ha='center', va='bottom', fontsize=8, rotation=0)

# Timeline
axes[1].plot(years, rates, 'o-', linewidth=2, markersize=10, color='blue')
for year, rate, name in zip(years, rates, names):
    axes[1].annotate(name.split()[0], (year, rate), textcoords="offset points", 
                    xytext=(5,5), ha='left', fontsize=8)

axes[1].set_xlabel('Year', fontsize=12)
axes[1].set_ylabel('Max Data Rate (Mbps)', fontsize=12)
axes[1].set_title('WiFi Data Rate Over Time', fontsize=14)
axes[1].set_yscale('log')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print comparison table
print("\nWiFi Standards Comparison:")
print("═" * 90)
print(f"{'Standard':<25} {'Year':<8} {'Freq':<15} {'Max Rate':<12} {'Modulation':<12} {'MIMO'}")
print("─" * 90)
for name, specs in wifi_standards.items():
    print(f"{name:<25} {specs['year']:<8} {specs['freq']:<15} {specs['max_rate']:<12} {specs['mod']:<12} {specs['mimo']}")
print("═" * 90)

# Calculate improvements
rate_11a = wifi_standards['802.11a']['max_rate']
rate_11ax = wifi_standards['802.11ax (WiFi 6)']['max_rate']
rate_11be = wifi_standards['802.11be (WiFi 7)']['max_rate']

print(f"\nKey Improvements:")
print(f"  WiFi 6 vs WiFi a: {rate_11ax/rate_11a:.1f}× faster")
print(f"  WiFi 7 vs WiFi a: {rate_11be/rate_11a:.1f}× faster")
print(f"  WiFi 7 vs WiFi 6: {rate_11be/rate_11ax:.1f}× faster")
print("\nKey Technologies:")
print("  • MIMO: Spatial multiplexing (multiple data streams)")
print("  • Wider channels: 20 → 40 → 80 → 160 → 320 MHz")
print("  • Higher modulation: 64-QAM → 256-QAM → 1024-QAM → 4096-QAM")
print("  • OFDMA: Multi-user efficiency (WiFi 6+)")
print("  • MU-MIMO: Serve multiple users simultaneously")

## Part 4: 5G NR and mmWave Analysis

**5G New Radio (NR)** operates in two frequency ranges:
- **FR1 (Sub-6 GHz)**: 450 MHz - 6 GHz (coverage)
- **FR2 (mmWave)**: 24-52 GHz (capacity)

### mmWave Challenges:
1. **High path loss**: ~30 dB more than 2.4 GHz at same distance
2. **Atmospheric absorption**: Rain, oxygen absorption
3. **Limited penetration**: Cannot pass through walls
4. **Solution**: Massive MIMO + Beamforming

In [ ]:
# 5G mmWave link budget example
print("5G mmWave Link Budget Analysis (28 GHz)")
print("═" * 70)

# Base parameters (without beamforming)
Pt_5g = 30  # dBm (1 W)
freq_5g = 28000  # MHz (28 GHz)
distance_5g = 0.1  # km (100 m)

# Scenario 1: Without beamforming (isotropic antennas)
Gt_iso = 0  # dBi
Gr_iso = 0  # dBi
Pr_iso, Lp_5g = link_budget(Pt_5g, Gt_iso, Gr_iso, distance_5g, freq_5g)

print("Scenario 1: Without Beamforming (Isotropic Antennas)")
print(f"  TX power: {Pt_5g} dBm")
print(f"  TX gain: {Gt_iso} dBi")
print(f"  RX gain: {Gr_iso} dBi")
print(f"  Path loss: {Lp_5g:.1f} dB")
print(f"  RX power: {Pr_iso:.1f} dBm")

# Scenario 2: With beamforming (massive MIMO)
Gt_bf = 20  # dBi (64-element array)
Gr_bf = 10  # dBi (16-element array)
Pr_bf, _ = link_budget(Pt_5g, Gt_bf, Gr_bf, distance_5g, freq_5g)

print("\nScenario 2: With Beamforming (Massive MIMO)")
print(f"  TX power: {Pt_5g} dBm")
print(f"  TX gain: {Gt_bf} dBi (beamforming)")
print(f"  RX gain: {Gr_bf} dBi (beamforming)")
print(f"  Path loss: {Lp_5g:.1f} dB")
print(f"  RX power: {Pr_bf:.1f} dBm")
print(f"\n  Beamforming gain: {Pr_bf - Pr_iso:.1f} dB")

# Compare to Sub-6 GHz
freq_sub6 = 2600  # MHz
Pr_sub6, Lp_sub6 = link_budget(Pt_5g, 0, 0, distance_5g, freq_sub6)

print("\nScenario 3: Sub-6 GHz Comparison (2.6 GHz)")
print(f"  TX power: {Pt_5g} dBm")
print(f"  TX gain: 0 dBi")
print(f"  RX gain: 0 dBi")
print(f"  Path loss: {Lp_sub6:.1f} dB")
print(f"  RX power: {Pr_sub6:.1f} dBm")

print("\n" + "═" * 70)
print("Analysis:")
print(f"  mmWave path loss penalty: {Lp_5g - Lp_sub6:.1f} dB vs Sub-6 GHz")
print(f"  Beamforming recovers: {Pr_bf - Pr_iso:.1f} dB")
print(f"  Net advantage of beamforming: {(Pr_bf - Pr_iso) - (Lp_5g - Lp_sub6):.1f} dB")
print("\n  Conclusion: Beamforming is ESSENTIAL for mmWave!")
print("═" * 70)

## Part 5: Link Budget Calculator for Different Scenarios

Let's create an interactive link budget analysis for various wireless systems.

In [ ]:
# Comprehensive link budget comparison
systems = {
    'WiFi 2.4 GHz (Home)': {
        'Pt': 20, 'Gt': 2, 'Gr': 0, 'freq': 2400, 'distance': 0.05,
        'margin': 10  # Walls, interference
    },
    'WiFi 5 GHz (Office)': {
        'Pt': 20, 'Gt': 5, 'Gr': 0, 'freq': 5000, 'distance': 0.03,
        'margin': 15  # More absorption at 5 GHz
    },
    'Cellular 4G (Urban)': {
        'Pt': 43, 'Gt': 15, 'Gr': 0, 'freq': 1800, 'distance': 1,
        'margin': 8  # Urban clutter
    },
    'Cellular 5G Sub-6 (Urban)': {
        'Pt': 43, 'Gt': 15, 'Gr': 0, 'freq': 3500, 'distance': 0.5,
        'margin': 10
    },
    '5G mmWave (Dense Urban)': {
        'Pt': 30, 'Gt': 25, 'Gr': 15, 'freq': 28000, 'distance': 0.1,
        'margin': 5  # Beamforming reduces effective margin
    },
    'Satellite GPS': {
        'Pt': 27, 'Gt': 13, 'Gr': 3, 'freq': 1575, 'distance': 20200,
        'margin': 2  # Space environment
    },
}

# Calculate link budgets
results = {}
for name, params in systems.items():
    Pr, Lp = link_budget(
        params['Pt'], params['Gt'], params['Gr'], 
        params['distance'], params['freq'], params['margin']
    )
    results[name] = {'Pr': Pr, 'Lp': Lp, **params}

# Print detailed results
print("\nComprehensive Link Budget Analysis:")
print("═" * 100)
print(f"{'System':<30} {'Pt (dBm)':<10} {'G_t (dBi)':<11} {'G_r (dBi)':<11} {'Loss (dB)':<11} {'Pr (dBm)'}")
print("─" * 100)

for name, res in results.items():
    total_loss = res['Lp'] + res['margin']
    print(f"{name:<30} {res['Pt']:<10} {res['Gt']:<11} {res['Gr']:<11} {total_loss:<11.1f} {res['Pr']:<.1f}")

print("═" * 100)

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

names = list(results.keys())
colors = plt.cm.tab10(np.arange(len(names)))

# Path loss comparison
path_losses = [results[n]['Lp'] for n in names]
axes[0, 0].barh(range(len(names)), path_losses, color=colors)
axes[0, 0].set_yticks(range(len(names)))
axes[0, 0].set_yticklabels(names, fontsize=9)
axes[0, 0].set_xlabel('Path Loss (dB)', fontsize=11)
axes[0, 0].set_title('Path Loss Comparison', fontsize=12)
axes[0, 0].grid(True, alpha=0.3, axis='x')

# Received power comparison
rx_powers = [results[n]['Pr'] for n in names]
bars = axes[0, 1].barh(range(len(names)), rx_powers, color=colors)
axes[0, 1].set_yticks(range(len(names)))
axes[0, 1].set_yticklabels(names, fontsize=9)
axes[0, 1].set_xlabel('Received Power (dBm)', fontsize=11)
axes[0, 1].set_title('Received Power Comparison', fontsize=12)
axes[0, 1].grid(True, alpha=0.3, axis='x')
axes[0, 1].axvline(x=-100, color='red', linestyle='--', linewidth=2, alpha=0.5, label='Typical sensitivity')
axes[0, 1].legend()

# Link margin (how much headroom)
# Assume typical sensitivity of -90 dBm for most systems
sensitivity = -90  # dBm (typical)
link_margins = [results[n]['Pr'] - sensitivity for n in names]

margin_colors = ['green' if m > 10 else 'orange' if m > 0 else 'red' for m in link_margins]
axes[1, 0].barh(range(len(names)), link_margins, color=margin_colors)
axes[1, 0].set_yticks(range(len(names)))
axes[1, 0].set_yticklabels(names, fontsize=9)
axes[1, 0].set_xlabel('Link Margin (dB)', fontsize=11)
axes[1, 0].set_title(f'Link Margin (Sensitivity = {sensitivity} dBm)', fontsize=12)
axes[1, 0].grid(True, alpha=0.3, axis='x')
axes[1, 0].axvline(x=0, color='red', linestyle='--', linewidth=2, label='Zero margin')
axes[1, 0].axvline(x=10, color='green', linestyle='--', linewidth=2, alpha=0.5, label='10 dB margin')
axes[1, 0].legend()

# Frequency vs distance scatter
frequencies = [results[n]['freq'] for n in names]
distances = [results[n]['distance'] * 1000 for n in names]  # Convert to meters

scatter = axes[1, 1].scatter(frequencies, distances, s=200, c=colors, alpha=0.7)
for i, name in enumerate(names):
    axes[1, 1].annotate(name.split()[0], (frequencies[i], distances[i]), 
                       fontsize=8, ha='left', va='bottom')

axes[1, 1].set_xlabel('Frequency (MHz)', fontsize=11)
axes[1, 1].set_ylabel('Distance (m)', fontsize=11)
axes[1, 1].set_title('Operating Frequency vs Range', fontsize=12)
axes[1, 1].set_xscale('log')
axes[1, 1].set_yscale('log')
axes[1, 1].grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.show()

print("\nKey Observations:")
print("  • GPS has highest path loss (20,200 km distance!)")
print("  • mmWave 5G requires beamforming to overcome high path loss")
print("  • WiFi 5 GHz has shorter range than 2.4 GHz")
print("  • Cellular uses high TX power and antenna gain for coverage")
print("  • Link margin > 10 dB is desired for reliable operation")

## Part 6: Coverage Range Estimation

Given a link budget, we can estimate the **maximum range** for different systems.

In [ ]:
def calculate_max_range(Pt_dBm, Gt_dBi, Gr_dBi, freq_MHz, sensitivity_dBm, margin_dB=10):
    """
    Calculate maximum range given link budget parameters
    
    Returns range in km
    """
    # Maximum allowable path loss
    max_path_loss = Pt_dBm + Gt_dBi + Gr_dBi - sensitivity_dBm - margin_dB
    
    # Solve Friis equation for distance
    # L_p = 20*log10(d) + 20*log10(f) + 32.45
    # d = 10^((L_p - 20*log10(f) - 32.45) / 20)
    
    d_km = 10**((max_path_loss - 20*np.log10(freq_MHz) - 32.45) / 20)
    
    return d_km, max_path_loss

# Calculate ranges for different systems
scenarios = [
    ('WiFi 2.4 GHz', 20, 2, 0, 2400, -90),
    ('WiFi 5 GHz', 20, 5, 0, 5000, -85),
    ('Cellular 4G LTE', 43, 15, 0, 1800, -110),
    ('5G Sub-6 GHz', 43, 15, 0, 3500, -105),
    ('5G mmWave (no BF)', 30, 0, 0, 28000, -80),
    ('5G mmWave (with BF)', 30, 25, 15, 28000, -80),
    ('LoRa (Sub-GHz)', 14, 3, 3, 915, -140),
]

print("\nMaximum Range Estimation:")
print("═" * 100)
print(f"{'System':<25} {'Pt':<8} {'Gt':<8} {'Gr':<8} {'Sens':<10} {'Max PL':<10} {'Range'}")
print("─" * 100)

ranges = []
for name, Pt, Gt, Gr, freq, sens in scenarios:
    max_range, max_pl = calculate_max_range(Pt, Gt, Gr, freq, sens, margin_dB=10)
    ranges.append(max_range)
    
    # Format range nicely
    if max_range < 1:
        range_str = f"{max_range*1000:.0f} m"
    else:
        range_str = f"{max_range:.2f} km"
    
    print(f"{name:<25} {Pt:<8} {Gt:<8} {Gr:<8} {sens:<10} {max_pl:<10.1f} {range_str}")

print("═" * 100)

# Visualize ranges
fig, ax = plt.subplots(figsize=(12, 8))

names_short = [s[0] for s in scenarios]
colors = plt.cm.tab10(np.arange(len(scenarios)))

# Convert to meters for better visualization
ranges_m = [r * 1000 for r in ranges]

bars = ax.barh(range(len(names_short)), ranges_m, color=colors)
ax.set_yticks(range(len(names_short)))
ax.set_yticklabels(names_short, fontsize=10)
ax.set_xlabel('Maximum Range (meters)', fontsize=12)
ax.set_title('Estimated Maximum Range for Different Wireless Systems\n(10 dB margin, free space)', fontsize=14)
ax.set_xscale('log')
ax.grid(True, alpha=0.3, axis='x', which='both')

# Add values on bars
for i, (bar, r_m) in enumerate(zip(bars, ranges_m)):
    if r_m < 1000:
        label = f"{r_m:.0f} m"
    else:
        label = f"{r_m/1000:.1f} km"
    ax.text(r_m * 1.1, i, label, va='center', fontsize=9)

plt.tight_layout()
plt.show()

print("\nKey Insights:")
print("  • LoRa: Longest range (>10 km) due to very high sensitivity (-140 dBm)")
print("  • Cellular 4G: Good coverage (~5 km) with high TX power and gain")
print("  • mmWave without beamforming: Very limited range (<100 m)")
print("  • mmWave with beamforming: Range increases 10× (~400 m)")
print("  • WiFi 5 GHz: Shorter range than 2.4 GHz due to higher frequency")
print("\nNote: These are theoretical free-space ranges.")
print("      Real-world ranges are typically 30-50% of these values.")

## Summary and Key Takeaways

### Link Budget Fundamentals
- **Friis Equation**: $P_r = P_t + G_t + G_r - L_p$
- **Path Loss**: Increases 20 dB/decade with distance and frequency
- **Link Margin**: Headroom above sensitivity (10 dB recommended)

### WiFi Evolution
- **802.11a/g**: 54 Mbps (1999-2003)
- **WiFi 4 (802.11n)**: 600 Mbps with 4×4 MIMO (2009)
- **WiFi 5 (802.11ac)**: 6.9 Gbps with 8×8 MIMO, 256-QAM (2014)
- **WiFi 6 (802.11ax)**: 9.6 Gbps with OFDMA, 1024-QAM (2019)
- **WiFi 7 (802.11be)**: 46 Gbps with 16×16 MIMO, 4096-QAM (2024)

### 5G NR Challenges
| Frequency Range | Coverage | Capacity | Challenge | Solution |
|----------------|----------|----------|-----------|----------|
| Sub-6 GHz | Excellent | Moderate | Spectrum congestion | Carrier aggregation |
| mmWave (24-52 GHz) | Poor | Excellent | High path loss | Massive MIMO + Beamforming |

### mmWave Path Loss Penalty
- **28 GHz vs 2.4 GHz**: ~21 dB more path loss at 100 m
- **Solution**: Beamforming gain of 30-40 dB (64-256 element arrays)
- **Result**: Range increases from 50 m to 500 m with beamforming

### Practical System Ranges (Free Space)
- **WiFi 2.4 GHz**: ~100 m
- **WiFi 5 GHz**: ~50 m
- **Cellular 4G**: ~5 km
- **5G Sub-6**: ~2 km
- **5G mmWave**: ~400 m (with beamforming)
- **LoRa**: >10 km (low data rate, high sensitivity)

### Design Trade-offs
1. **Frequency**: Higher → More path loss → Shorter range
2. **Bandwidth**: Wider → Higher data rate → More capacity
3. **Modulation**: Higher order → More bits/symbol → Requires higher SNR
4. **MIMO**: More antennas → Higher throughput → More complexity
5. **Beamforming**: Essential for mmWave to overcome path loss

### Future Trends
- **6G**: Terahertz (>100 GHz), even more beamforming needed
- **Reconfigurable Intelligent Surfaces (RIS)**: Reflect/focus signals
- **AI-based optimization**: Adaptive beamforming and resource allocation
- **Integrated sensing and communication**: Radar + communication